##### Bibliotecas

In [ ]:
import pandas as pd
import pathlib
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email import encoders
import os

##### Base de Dados

In [ ]:
emails = pd.read_excel(r'Bases de Dados\Emails.xlsx')
lojas = pd.read_csv(r'Bases de Dados\Lojas.csv', sep=';', encoding='latin1')
vendas = pd.read_excel(r'Bases de Dados\Vendas.xlsx')

display(emails)
display(lojas)
display(vendas)

##### Criar uma Tabela para cada Loja e Definir o dia do Indicador

In [ ]:
vendas = vendas.merge(lojas, on='ID Loja')
display(vendas)

In [ ]:
dicionario_lojas = {}

for loja in lojas['Loja']:
  dicionario_lojas[loja] = vendas.loc[vendas['Loja']==loja, :]
  
display(dicionario_lojas['Iguatemi Esplanada'])
display(dicionario_lojas['Shopping Vila Velha'])

##### Definindo o Dia do Indicador

In [ ]:
dia_indicador = vendas['Data'].max()
print(dia_indicador)
print(f'{dia_indicador.day}/{dia_indicador.month}')

##### Salvar a planilha na pasta de Backup

In [ ]:
caminho_backup = pathlib.Path(r'Backup Arquivos Lojas')

arquivos_pasta_backup = caminho_backup.iterdir()
lista_nomes_backup = [arquivo.name for arquivo in arquivos_pasta_backup]

for loja in dicionario_lojas:
    if loja not in lista_nomes_backup:
        nova_pasta = caminho_backup / loja
        nova_pasta.mkdir()

    nome_arquivo = '{}_{}_{}.xlsx'.format(dia_indicador.month, dia_indicador.day, loja)
    local_arquivo = caminho_backup / loja / nome_arquivo

    dicionario_lojas[loja].to_excel(local_arquivo)

##### Calcular o indicador para 1 loja

In [ ]:
loja = 'Norte Shopping'
vendas_lojas = dicionario_lojas[loja]
vendas_lojas_dia = vendas_lojas.loc[vendas_lojas['Data']==dia_indicador, :]

- Faturamento

In [ ]:
faturamento_ano = vendas_lojas['Valor Final'].sum()
print(faturamento_ano)

faturamento_dia = vendas_lojas_dia['Valor Final'].sum()
print(faturamento_dia)

- Diversidade de Produtos

In [ ]:
qtde_ano = len(vendas_lojas['Produto'].unique())
print(qtde_ano)

qtde_dia = len(vendas_lojas_dia['Produto'].unique())
print(qtde_dia)

- Ticket Médio

In [ ]:
valor_venda = vendas_lojas.groupby('Código Venda').sum()
ticket_medio_ano = valor_venda['Valor Final'].mean()
print(ticket_medio_ano)

valor_venda_dia = vendas_lojas_dia.groupby('Código Venda').sum()
ticket_medio_dia = valor_venda_dia['Valor Final'].mean()
print(ticket_medio_dia)

- Meta

In [ ]:
meta_faturamento_dia = 1000
meta_faturamento_ano = 1650000
meta_qtdeprodutos_dia = 4
meta_qtdeprodutos_ano = 120
meta_ticketmedio_dia = 500
meta_ticketmedio_ano = 500

##### Enviar por e-mail para o gerente

In [ ]:
for loja in dicionario_lojas:

    vendas_loja = dicionario_lojas[loja]
    vendas_loja_dia = vendas_loja.loc[vendas_loja['Data']==dia_indicador, :]

    #faturamento
    faturamento_ano = vendas_loja['Valor Final'].sum()
    #print(faturamento_ano)
    faturamento_dia = vendas_loja_dia['Valor Final'].sum()
    #print(faturamento_dia)

    #diversidade de produtos
    qtde_produtos_ano = len(vendas_loja['Produto'].unique())
    #print(qtde_produtos_ano)
    qtde_produtos_dia = len(vendas_loja_dia['Produto'].unique())
    #print(qtde_produtos_dia)


    #ticket medio
    valor_venda = vendas_loja.groupby('Código Venda').sum()
    ticket_medio_ano = valor_venda['Valor Final'].mean()
    #print(ticket_medio_ano)
    #ticket_medio_dia
    valor_venda_dia = vendas_loja_dia.groupby('Código Venda').sum()
    ticket_medio_dia = valor_venda_dia['Valor Final'].mean()
    #print(ticket_medio_dia)

    # Configurações do email

    nome = emails.loc[emails['Loja']==loja, 'Gerente'].values[0]
    fromaddr = "gsoaresdesouzaaws@gmail.com"
    toaddr = emails.loc[emails['Loja']==loja, 'E-mail'].values[0]
    subject = f'OnePage Dia {dia_indicador.day}/{dia_indicador.month} - Loja {loja}'

    if faturamento_dia >= meta_faturamento_dia:
        cor_fat_dia = 'green'
    else:
        cor_fat_dia = 'red'
    if faturamento_ano >= meta_faturamento_ano:
        cor_fat_ano = 'green'
    else:
        cor_fat_ano = 'red'
    if qtde_produtos_dia >= meta_qtdeprodutos_dia:
        cor_qtde_dia = 'green'
    else:
        cor_qtde_dia = 'red'
    if qtde_produtos_ano >= meta_qtdeprodutos_ano:
        cor_qtde_ano = 'green'
    else:
        cor_qtde_ano = 'red'
    if ticket_medio_dia >= meta_ticketmedio_dia:
        cor_ticket_dia = 'green'
    else:
        cor_ticket_dia = 'red'
    if ticket_medio_ano >= meta_ticketmedio_ano:
        cor_ticket_ano = 'green'
    else:
        cor_ticket_ano = 'red'

    body = f'''
      <p>Bom dia, {nome}</p>

      <p>O resultado de ontem <strong>({dia_indicador.day}/{dia_indicador.month})</strong>, da <strong>Loja {loja}</strong> foi:</p>

      <table>
        <tr>
          <th>Indicador</th>
          <th>Valor Dia</th>
          <th>Meta Dia</th>
          <th>Cenário Dia</th>
        </tr>
        <tr>
          <td>Faturamento</td>
          <td style="text-align: center">R$ {faturamento_dia:.2f}</td>
          <td style="text-align: center">R$ {meta_faturamento_dia:.2f}</td>
          <td style="text-align: center"><font color="{cor_fat_dia}">◙</font></td>
        </tr>
        <tr>
          <td>Diversidade de Produtos</td>
          <td style="text-align: center">{qtde_produtos_dia}</td>
          <td style="text-align: center">{meta_qtdeprodutos_dia}</td>
          <td style="text-align: center"><font color="{cor_qtde_dia}">◙</font></td>
        </tr>
        <tr>
          <td>Ticket Médio</td>
          <td style="text-align: center">R$ {ticket_medio_dia:.2f}</td>
          <td style="text-align: center">R$ {meta_ticketmedio_dia:.2f}</td>
          <td style="text-align: center"><font color="{cor_ticket_dia}">◙</font></td>
        </tr>
      </table>
        <br>
      <table>
        <tr>
          <th>Indicador</th>
          <th>Valor Ano</th>
          <th>Meta Ano</th>
          <th>Cenário Ano</th>
        </tr>
        <tr>
          <td>Faturamento</td>
          <td style="text-align: center">R$ {faturamento_ano:.2f}</td>
          <td style="text-align: center">R$ {meta_faturamento_ano:.2f}</td>
          <td style="text-align: center"><font color="{cor_fat_ano}">◙</font></td>
        </tr>
        <tr>
          <td>Diversidade de Produtos</td>
          <td style="text-align: center">{qtde_produtos_ano}</td>
          <td style="text-align: center">{meta_qtdeprodutos_ano}</td>
          <td style="text-align: center"><font color="{cor_qtde_ano}">◙</font></td>
        </tr>
        <tr>
          <td>Ticket Médio</td>
          <td style="text-align: center">R$ {ticket_medio_ano:.2f}</td>
          <td style="text-align: center">R$ {meta_ticketmedio_ano:.2f}</td>
          <td style="text-align: center"><font color="{cor_ticket_ano}">◙</font></td>
        </tr>
      </table>

      <p>Seque em anexo a planilha com todos os dados para mais detalhes.</p>

      <p>Qualquer dúvida estou à disposição</p>
      <p>Att.; Gabriel</p>
    '''

    # Configurações do anexo
    filename = f'{dia_indicador.month}_{dia_indicador.day}_{loja}.xlsx'
    filepath = os.path.join(pathlib.Path.cwd(), caminho_backup, loja, filename)

    # Configuração da mensagem
    msg = MIMEMultipart()
    msg['From'] = fromaddr
    msg['To'] = toaddr
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'html'))

    # Configuração do anexo
    attachment = open(filepath, "rb")
    part = MIMEBase('application', 'octet-stream')
    part.set_payload(attachment.read())
    encoders.encode_base64(part)
    part.add_header('Content-Disposition', "attachment; filename= %s" % filename)

    msg.attach(part)

    # Enviando o email
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login(fromaddr, "llsmaiefjhecvqxq")
    text = msg.as_string()
    server.sendmail(fromaddr, toaddr, text)
    server.quit()

    print('E-mail da Loja {} enviado'.format(loja))

### Passo 6 - Automatizar todas as lojas

### Passo 7 - Criar ranking para diretoria

### Passo 8 - Enviar e-mail para diretoria